In [1]:
import sys;
import py_ecc.bn128
from py_ecc.bn128 import G1, G2, multiply, add, curve_order, eq, neg, pairing, Z1, Z2
import galois
from functools import reduce
import numpy as np
from operator import add


def evaluate_polynomial_galois(coefs, x, GF):
    result = GF(0)
    for coef in coefs:
        result = result * x + GF(coef)
    return int(result)

def inner_product_polynomials_with_witness(polys, witness):
    mul_ = lambda x, y: x * y
    sum_ = lambda x, y: x + y
    return reduce(sum_, map(mul_, polys, witness))

In [2]:
GF = galois.GF(curve_order)

In [20]:
## TRUSTED SETUP
def trusted_setup(U,V,W,l):

    nb_eq = len(U)

    #=================================Pour trouver t(x)  ==========================
    values = [i for i in range(1, len(U[1])+1)] # Car on a 6 colonnes
    poly_t = galois.Poly([1], field=GF)
    for val in values:
        poly_t *= galois.Poly([1, -val], field=GF)
    t=[]
    for elem in poly_t.coeffs:
        t.append(int(elem))
    print("t=",t)
    #==============================================================================

    x=GF(13)
    t_x=evaluate_polynomial_galois(t, x, GF)

    alpha=GF(5)
    beta=GF(7)

    x_power_i=[]
    for i in range(0, nb_eq):
        x_power_i.append(x**i)

    #Transfomation des matrices en liste de polynomes
    U_polys=[]
    V_polys=[]
    W_polys=[]

    for elem in U:
        U_polys.append(galois.Poly((elem),field=GF))
        V_polys.append(galois.Poly((elem),field=GF))
        W_polys.append(galois.Poly((elem),field=GF))


    polys=[]
    for i in range(0, nb_eq): 
        u_i=U_polys[i](x**i)
        v_i=V_polys[i](x**i)
        w_i=W_polys[i](x**i)
        polys.append((int(beta)*u_i+int(alpha)*v_i+w_i))

    x_power_i_t_x=[]
    for i in range(nb_eq-1):
        x_power_i_t_x.append((x**i)*t_x)

    G1_alpha = multiply(G1, int(alpha))
    G2_beta = multiply(G2, int(beta))

    G1_x_power_i = [multiply(G1,int(elem)) for elem in x_power_i]
    G2_x_power_i = [multiply(G2,int(elem)) for elem in x_power_i]

    G1_power_x = [multiply(G1,int(elem)) for elem in polys]
    G1_x_power_i_t_x = [multiply(G1,int(elem)) for elem in x_power_i_t_x]

    output=[G1_x_power_i,G1_alpha,G2_x_power_i,G2_beta,G1_power_x,G1_x_power_i_t_x]
    return output

In [28]:
## PROVER
def prover(U,V,W,l,keys,a):
    print("\nProver")

    m = len(a)
    G1_x_power_i,G1_alpha,G2_x_power_i,G2_beta,G1_power_x,G1_x_power_i_t_x = keys

    #=================================Pour trouver t(x)  ==========================
    values = [i for i in range(1, len(U[1])+1)] # Car on a 6 colonnes

    poly_t = galois.Poly([1], field=GF)  # Start with the polynomial '1' in the given field
    for val in values:
        poly_t *= galois.Poly([1, -val], field=GF)
    t=[]
    for elem in poly_t.coeffs:
        t.append(int(elem))

    print("t=",t)
    #=================================Pour trouver h(x)  ==========================
    U_polys=[]
    V_polys=[]
    W_polys=[]
    for i in range(len(U)):
        U_polys.append(galois.Poly((U[i]),field=GF))
        V_polys.append(galois.Poly((V[i]),field=GF))
        W_polys.append(galois.Poly((W[i]),field=GF))

    witness = np.array(a) % curve_order

    Ua = inner_product_polynomials_with_witness(U_polys, witness)
    Va = inner_product_polynomials_with_witness(V_polys, witness)
    Wa = inner_product_polynomials_with_witness(W_polys, witness)

    t_poly=galois.Poly(t,field=GF)

    h = (Ua * Va - Wa) // t_poly
    h_rem = (Ua * Va - Wa) % t_poly

    assert h_rem == 0, "h(x) is not a multiple of t(x)"

    h_t_x=None

    for i in range(0, len(h.coeffs)):
        h_t_x=py_ecc.bn128.add(h_t_x,multiply(G1_x_power_i_t_x[i],int(h.coeffs[i])))
    print("h_t_x=",h_t_x)


    #==============================================================================

    Ui_x = []
    for i in range(0, m):
        temp=None
        for j in range(0, len(U[i])):
            temp2=multiply(G1_x_power_i[j],int(U[i][j]))
            temp=py_ecc.bn128.add(temp,temp2)
        Ui_x.append(temp)

    A_1=None
    for i in range(0, m):
        temp=multiply(Ui_x[i],int(a[i]))
        A_1=py_ecc.bn128.add(A_1,temp)
    A_1=py_ecc.bn128.add(A_1,G1_alpha)
    print("A=",A_1)

    V_i_x = []
    for i in range(0, m):
        temp=None
        for j in range(0, len(V[i])):
            temp2=multiply(G2_x_power_i[j],int(V[i][j]))
            temp=py_ecc.bn128.add(temp,temp2)
        V_i_x.append(temp)

    B_2=None
    for i in range(0, m):
        temp=multiply(V_i_x[i],int(a[i]))
        B=py_ecc.bn128.add(B_2,temp)
    B_2=py_ecc.bn128.add(B_2,G2_beta)
    print("B=",B_2)

    C_1=None
    print(G1_power_x)
    print(a)
    for i in range(0, m):
        temp=multiply(G1_power_x[i],int(a[i]))
        C_1=py_ecc.bn128.add(C_1,temp)
    C_1=py_ecc.bn128.add(C_1,h_t_x)

    return A_1, B_2, C_1
    # A = alpha_G1 + multiply(delta_G1,r)
    # for i in range(0, m):
    #     A+= a[i] * (U_polys[i](tau))
    #     A= add(A, multiply(x_power_i_G1[i],s))

    # C_1=GF(0)
    # for i in range(2, m):
    #     C_1+=a[i]*(beta * (U_polys[i](tau)) + alpha * (V_polys[i](tau)) + W_polys[i](tau))
    # C_1=C_1 / delta
    # C_2=(h_quo(tau) * t_poly(tau)) / delta
    # C_3=(A * s + B * r - r * s * delta)
    # C=C_1+C_2+C_3
    # print("C_bis=",C)

    # A_1=multiply(G1,int(A)) 
    # C_1=multiply(G1,int(C))
    # B_2=multiply(G2,int(B))

    # print("A_1=",A_1)
    # print("C_1=",C_1)
    # print("B_2=",B_2)

    #Le prover va donner A_1, C_1 et B_2 

In [29]:
## VERIFIER

def verifier(A, C, B,keys,a):
    print("\nVerifier")
    m = len(a)

    G1_x_power_i,G1_alpha,G2_x_power_i,G2_beta,G1_power_x,G1_x_power_i_t_x = keys

    left=pairing(B,A)
    right_1=pairing(G2_beta,G1_alpha)
    right_2=pairing(G2,C)

    print("left=",left)
    print("right_1 =",right_1)
    print("right_2 =",right_2)

    print(left == right_1 + right_2)
    # D=GF(0)
    # for i in range(0, 2):
    #     D+=a[i]*(beta * (U_polys[i](tau)) + alpha * (V_polys[i](tau)) + W_polys[i](tau))
    # D=multiply(G1,int(D/gamma))
    # print("D_1= ",D)

    # lhs = pairing(B_2, A_1) #A_1*B_2
    # print("lhs=",lhs)

    # rhs_1 = pairing(multiply(G2,int(beta)),multiply(G1,int(alpha)))
    # rhs_2 = pairing(multiply(G2,int(gamma)),D)
    # rhs_3 = pairing(multiply(G2,int(delta)),C_1)
    # rhs=rhs_1*rhs_2*rhs_3
    # print("rhs=",rhs)
    # print(lhs == rhs)


In [30]:
## DATA 


def main():
    # x^3+x+5
    # Ensemble des varibles = ['1', 'out', 'x', 'v1', 'v2', 'v3']
    U=[
        [3648040478639879203707734290876212514758060733402672390616367364429301415937, 21888242871839275222246405745257275088548364400416034343698204186575808495612, 18240202393199396018538671454381062573790303667013361953081836822146507079690, 21888242871839275222246405745257275088548364400416034343698204186575808495612], 
        [0, 0, 0, 0], 
        [14592161914559516814830937163504850059032242933610689562465469457717205663744, 5, 7296080957279758407415468581752425029516121466805344781232734728858602831861, 8], 
        [10944121435919637611123202872628637544274182200208017171849102093287904247809, 21888242871839275222246405745257275088548364400416034343698204186575808495613, 10944121435919637611123202872628637544274182200208017171849102093287904247818, 21888242871839275222246405745257275088548364400416034343698204186575808495611], 
        [10944121435919637611123202872628637544274182200208017171849102093287904247808, 10944121435919637611123202872628637544274182200208017171849102093287904247812, 21888242871839275222246405745257275088548364400416034343698204186575808495610, 4], 
        [18240202393199396018538671454381062573790303667013361953081836822146507079681, 21888242871839275222246405745257275088548364400416034343698204186575808495616, 3648040478639879203707734290876212514758060733402672390616367364429301415938, 21888242871839275222246405745257275088548364400416034343698204186575808495616]]

    V=[
        [7296080957279758407415468581752425029516121466805344781232734728858602831872, 10944121435919637611123202872628637544274182200208017171849102093287904247811, 3648040478639879203707734290876212514758060733402672390616367364429301415931, 3], 
        [0, 0, 0, 0], 
        [14592161914559516814830937163504850059032242933610689562465469457717205663745, 10944121435919637611123202872628637544274182200208017171849102093287904247806, 18240202393199396018538671454381062573790303667013361953081836822146507079686, 21888242871839275222246405745257275088548364400416034343698204186575808495615], 
        [0, 0, 0, 0], 
        [0, 0, 0, 0], 
        [0, 0, 0, 0]]

    W=[
        [0, 0, 0, 0], 
        [18240202393199396018538671454381062573790303667013361953081836822146507079681, 21888242871839275222246405745257275088548364400416034343698204186575808495616, 3648040478639879203707734290876212514758060733402672390616367364429301415938, 21888242871839275222246405745257275088548364400416034343698204186575808495616], 
        [0, 0, 0, 0], 
        [3648040478639879203707734290876212514758060733402672390616367364429301415936, 10944121435919637611123202872628637544274182200208017171849102093287904247810, 7296080957279758407415468581752425029516121466805344781232734728858602831868, 4], 
        [10944121435919637611123202872628637544274182200208017171849102093287904247809, 21888242871839275222246405745257275088548364400416034343698204186575808495613, 10944121435919637611123202872628637544274182200208017171849102093287904247818, 21888242871839275222246405745257275088548364400416034343698204186575808495611], 
        [10944121435919637611123202872628637544274182200208017171849102093287904247808, 10944121435919637611123202872628637544274182200208017171849102093287904247812, 21888242871839275222246405745257275088548364400416034343698204186575808495610, 4]
    ]

    W=GF(W)
    U=GF(U)
    V=GF(V)



    a=[1, 135, 5, 25, 125, 130]
    a=GF(a)

    l=2 # car dans a=[1, 135, 5, 25, 125, 130] on donne en public les 2 premiers éléments de a et le reste c'est privée
    keys= trusted_setup(U,V,W,l)

    print("Sigma_1")
    for elem in keys:
        print(elem)
    print(" ")

    A,B,C = prover(U,V,W,l,keys,a)

    print("A=",A)
    print("B=",B)
    print("C=",C)
    
    verifier(A, C, B,keys,a)

    


main()


t= [1, 21888242871839275222246405745257275088548364400416034343698204186575808495607, 35, 21888242871839275222246405745257275088548364400416034343698204186575808495567, 24]
Sigma_1
[(1, 2), (2672242651313367459976336264061690128665099451055893690004467838496751824703, 18247534626997477790812670345925575171672701304065784723769023620148097699216), (12787339590274394701415274422371285504222742952103028517882107571519658525136, 4739954905410646502518070840395475123473740446172208598017525809409688232767), (11196867986020524887868151181298169377551861207909865023458178113072991099778, 14907424744721644019295601827429271389033647158022784104065095546615339379684), (1135991510683317975222100808031595125091948809584947113331576335667932330104, 14149795409294089364136043915968488764428369360532124243297730273093312337883), (3800731997910183889945081371365748011254984353065867732853987733873756390398, 3533424467713456872660920663164357821561100952540133840212070767351347700165)]
(10744596414106

In [31]:

#Le prouver gen r,s deux randoms pour avoir du ZKP
r=GF(30)
s=GF(200)

h_tau=evaluate_polynomial_galois(h_quo.coeffs, tau, GF)
ht_tau=h_tau*t_tau
print("ht(tau)=",ht_tau)

#===============================CALCUL DE A============================================
A_G1=None
A_0=alpha_G1

A_1=None
for i in range(0, nb_eq):
    eval_U_tau=evaluate_polynomial_galois(U[i], tau, GF)
    eval_U_tau_G1=multiply(G1, eval_U_tau)
    eval_aU_tau_G1=multiply(eval_U_tau_G1, int(a[i]))
    A_1=add(A_1, eval_aU_tau_G1)

A_2=multiply(delta_G1, int(r))

A_G1=add(A_0, add(A_1, A_2))
print("A_G1=",A_G1)

#===============================CALCUL DE B_2============================================
B_G2=None
B_0=beta_G2

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G2=multiply(G2, eval_V_tau)
    eval_aV_tau_G2=multiply(eval_V_tau_G2, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G2)

B_2=multiply(delta_G2, int(s))

B_G2=add(B_0, add(B_1, B_2))
print("B_G2=",B_G2)

#===============================CALCUL DE B_1============================================
B_G1=None
B_0=beta_G1

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G1=multiply(G1, eval_V_tau)
    eval_aV_tau_G1=multiply(eval_V_tau_G1, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G1)

B_2=multiply(delta_G1, int(s))

B_G1=add(B_0, add(B_1, B_2))
print("B_G1=",B_G1)

#===============================CALCUL DE C ============================================
C=None

C_0=None
for i in range(0, nb_eq):
    eval_U_tau=evaluate_polynomial_galois(U[i], tau, GF)
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_W_tau=evaluate_polynomial_galois(W[i], tau, GF)
    temp=int(beta)*int(eval_U_tau)+int(alpha)*int(eval_V_tau)+int(eval_W_tau)
    eval_G1=multiply(G1, temp)
    eval_aW_tau_G1=multiply(eval_G1, int(a[i]))
    C_0=add(C_0, eval_aW_tau_G1)
C_1=None
C_2=None
C_3=None
C_4=None

NameError: name 'h_quo' is not defined

In [ ]:

#Le prouver gen r,s deux randoms pour avoir du ZKP
r=GF(30)
s=GF(200)

h_tau=evaluate_polynomial_galois(h_quo.coeffs, tau, GF)
ht_tau=h_tau*t_tau
print("ht(tau)=",ht_tau)

#===============================CALCUL DE A============================================
A_G1=None
A_0=int(alpha)

A_1=0
for i in range(0, nb_eq):
    A_1+=int(a[i])*evaluate_polynomial_galois(U[i], tau, GF)

A_2=int(r)*int(delta)

A_G1=A_0+A_1+A_2
print("A_G1=",A_G1)
print("A_G1=",multiply(G1,A_G1))


#===============================CALCUL DE B_2============================================
B_G2=None
B_0=beta_G2

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G2=multiply(G2, eval_V_tau)
    eval_aV_tau_G2=multiply(eval_V_tau_G2, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G2)

B_2=multiply(delta_G2, int(s))

B_G2=add(B_0, add(B_1, B_2))
print("B_G2=",B_G2)

#===============================CALCUL DE B_1============================================
B_G1=None
B_0=beta_G1

B_1=None
for i in range(0, nb_eq):
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_V_tau_G1=multiply(G1, eval_V_tau)
    eval_aV_tau_G1=multiply(eval_V_tau_G1, int(a[i]))
    B_1=add(B_1, eval_aV_tau_G1)

B_2=multiply(delta_G1, int(s))

B_G1=add(B_0, add(B_1, B_2))
print("B_G1=",B_G1)

#===============================CALCUL DE C ============================================
C=None

C_0=None
for i in range(0, nb_eq):
    eval_U_tau=evaluate_polynomial_galois(U[i], tau, GF)
    eval_V_tau=evaluate_polynomial_galois(V[i], tau, GF)
    eval_W_tau=evaluate_polynomial_galois(W[i], tau, GF)
    temp=int(beta)*int(eval_U_tau)+int(alpha)*int(eval_V_tau)+int(eval_W_tau)
    eval_G1=multiply(G1, temp)
    eval_aW_tau_G1=multiply(eval_G1, int(a[i]))
    C_0=add(C_0, eval_aW_tau_G1)
C_1=None
C_2=None
C_3=None
C_4=None